In [ ]:
#needs to be executed in terminal
pip install fuzzywuzzy
pip install python-Levenshtein

In [3]:
import re
import pandas as pd
import numpy as np
from pathlib import Path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Functions

In [5]:
#Code snippet: https://programminghistorian.org/en/lessons/generating-an-ordered-data-set-from-an-OCR-text-file

'''
Code ripped from https://www.datacamp.com/community/tutorials/fuzzy-string-python
'''
def lev(seq1, seq2):
    """ levenshtein_ratio_and_distance:
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of seq1 and the
        first j characters of seq2
    """
    # Initialize matrix of zeros
    rows = len(seq1)+1
    cols = len(seq2)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if seq1[row-1] == seq2[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions

    return distance[row][col]

# Code

## GEG

### Section Tagging

##### Manual changes:  
- In file 1873_GEG_New.txt I manually removed additional spaces from line 632 and line 1626.  
- In file 1874_GEG_New.txt I manually removed additional spaces from line 3747 and line 5449 and line 6234.  
- In file 1875_GEG_New.txt I manually removed additional spaces from line 957 and line 5255.  
- In file 1875_GEG_New.txt I manually added a line break in line 6016 after "Verzeichnis ".
- In file 1876_GEG_New.txt I manually removed additional spaces from line 7158.
- In file 1876_GEG_New.txt I manually added a line break in line 8200 after "Verzeichnis ".
- In file 1877_GEG_New.txt I manually removed additional spaces from line 1790.
- In file 1877_GEG_New.txt I manually added a line break in line 5130 after "Verzeichnis ".
- In file 1878_GEG_New.txt I manually removed additional symbols bevor and after the content from line 5052.
- In file 1878_GEG_New.txt I manually added a line break in line 8490 after "Verzeichnis ".
- In file 1879_GEG_New.txt I manually added a line break in line 7390 after "Verzeichnis ".
- In file 1880_GEG_New.txt I manually added a line break in line 6022 after "Verzeichnis ".
- In file 1881_GEG_New.txt I manually added a line break in line 7500 after "Verzeichnis ".
- In file 1882_GEG_New.txt I manually added a line break in line 4971 to move the "Sitzung" in a new line.
- In file 1882_GEG_New.txt I manually added a line break in line 7860 after "Verzeichnis ".
- In file 1882_GEG_New.txt I manually removed additional spaces from line 8424.
- In file 1883_GEG_New.txt I manually added a line break in line 5306 after "Verzeichnis ".
- In file 1884_GEG_New.txt I manually added a line break in line 9822 after "Verzeichnis ".
- In file 1885_GEG_New.txt I manually added a line break in line 10083 after "Verzeichnis ".
- In file 1886_GEG_New.txt I manually added a line break in line 9575 after "Verzeichnis ".
- In file 1887_GEG_New.txt I manually added a line break in line 4721 after "Verzeichnis ".
- In file 1887_GEG_New.txt I manually removed additional spaces from line 5122 and line 5367 and line 5625.
- In file 1888_GEG_New.txt I manually added "Jahresversammlung" and a line break in line 72.
- In file 1888_GEG_New.txt I manually added a line break in line 8039 after "Verzeichnis ".
- In file 1889_GEG_New.txt I manually added a line break in line 5197 after "Verzeichnis ".
- In file 1890_GEG_New.txt I manually added a line break in line 3908 after "Verzeichnis ".
- In file 1891_GEG_New.txt I manually added a line break in line 4752 after "Verzeichnis ".
- In file 1892_GEG_New.txt I manually added a line break in line 3944 after "Verzeichnis ".
- NB: File 1892_GEG_New.txt has missing numbers for the members-list between line 4142 and line 4221.
- In file 1893_GEG_New.txt I manually added a line break in line 4384 after "Verzeichnis ".
- In file 1894_GEG_New.txt I manually added a line break in line 6047 after "Verzeichnis ".
- NB: File 1894_GEG_New.txt has missing numbers for the members-list between line 6180 and line 6212.
- In file 1895_GEG_New.txt I manually added a line break in line 4303 after "Verzeichnis ".
- In file 1896_GEG_New.txt I manually added a line break in line 5780 after "Verzeichnis ".
- In file 1896_GEG_New.txt I manually removed additional spaces from line 6212 and line 6463.
- In file 1897_GEG_New.txt I manually added a line break in line 6221 after "Verzeichnis ".
- In file 1898_GEG_New.txt I manually added a line break in line 4560 after "Verzeichnis ".
- In file 1899_GEG_New.txt I manually added a line break in line 4992 after "Verzeichnis ".
- In file 1900_GEG_New.txt I manually added a line break in line 6014 after "Verzeichnis ".
- NB: File 1900_GEG_New.txt has missing numbers for the members-list between line 6191 and line 6225.
- In file 1901_GEG_New.txt I manually added a line break in line 9401 after "Verzeichnis ".
- In file 1901_GEG_New.txt I manually removed a line break in line 7995 after "Verzeichnis ".
- In file 1902_GEG_New.txt I manually added a line break in line 6192 after "Verzeichnis ".
- In file 1903_GEG_New.txt I manually added a line break in line 100, line 240, line 466, line 534, line 686, and line 785 after "Sitzung".
- In file 1903_GEG_New.txt I manually added a line break in line 1279 after "Verzeichnis ".
- In file 1903_GEG_New.txt I manually removed a line break in line 6357 after "Verzeichnis ".
- In file 1904_GEG_New.txt I manually added a line break in line 224, line 240, line 389, line 446, line 910, line 1087, line 1198, and line 1335 after "Sitzung".
- In file 1904_GEG_New.txt I manually added a line break in line 1578 after "Verzeichnis ".
- NB: File 1904_GEG_New.txt has missing numbers for the members-list between line 1765 and line 1798.
- NB: File 1904_GEG_New.txt has missing numbers for the members-list between line 1845 and line 1878.
- In file 1905_GEG_New.txt I manually added a line break in line 90, line 208, line 270, line 581, line 679, line 966, line 1215, and line 1278 after "Sitzung".
- In file 1905_GEG_New.txt I manually added a line break in line 1470 after "Verzeichnis ".
- In file 1906_GEG_New.txt I manually added a line break in line 68, line 112, line 222, line 339, line 513, line 632, and line 703 after "Sitzung".
- In file 1906_GEG_New.txt I manually added a line break in line 881 after "Verzeichnis ".
- In file 1907_GEG_New.txt I manually added a line break in line 99, line 168, line 209, line 275, line 433, line 537, line 890, line 951, and line 1084 after "Sitzung".
- In file 1907_GEG_New.txt I manually added a line break in line 1386 after "Verzeichnis ".
- In file 1908_GEG_New.txt I manually added a line break in line 97, line 245, line 319, line 449, line 614, line 851, line 1045, and line 1113 after "Sitzung".
- In file 1908_GEG_New.txt I manually added a line break in line 1500 after "Verzeichnis ".
- In file 1909_GEG_New.txt I manually added a line break in line 85, line 486, line 628, line 762, line 898, line 1281, and line 1367 after "Sitzung".
- In file 1909_GEG_New.txt I manually added a line break in line 1651 after "Verzeichnis ".
- In file 1910_GEG_New.txt I manually added a line break in line 93, line 254, line 336, line 389, line 458, line 529, line 579 and line 604 after "Sitzung".
- In file 1910_GEG_New.txt I manually added a line break in line 731 after "Verzeichnis ".
- NB: File 1910_GEG_New.txt has missing numbers for the members-list between line 760 and line 790.
- In file 1911_GEG_New.txt I manually added a line break in line 86, line 226, line 301, line 389, line 520, line 578, and line 639 after "Sitzung".
- In file 1911_GEG_New.txt I manually added a line break in line 3742 and in line 4552 after "Verzeichnis ".
- NB: In file 1911_GEG_New.txt the last "Verzeichnis" is the membership list. 
- In file 1912-1922_GEG_New.txt I manually added a line break in line 3167 after "Sitzung" and replaced "Außerordentliche Sitzung" with "XXX. Monatssitzung"
- In file 1912-1922_GEG_New.txt I manually added a line break in line 72, line 349, line 392, line 3249, line 3364, line 4247, line 4267, line 4378, line 4585, line 4777, line 4904, line 5072, line 5187, and line 5294 after "Monatssitzung".
- In file 1912-1922_GEG_New.txt I manually added a line break in line 3742 and in line 8407 after "Verzeichnis ".
- NB: In file 1912-1922_GEG_New.txt the membership list does have a completely different layout. 

In [92]:
#I read the file into a list
filename = 'Sources/GEG/1912-1922_GEG_New.txt'
with open(filename, 'r') as fp:
    text = fp.readlines()
    

In [93]:
#I check whether all the sections I want to tag are being detected. 
#If I do notice missing sections I have to check the file manually and adapt the lines if needed.

for num, line in enumerate(text, 1):
    
    yearlymeet_lev_score = lev(line,'Jahresversammlung \n')                            
    section_lev_score = lev(line, '750. Monatssitzung \n')                                    #I adjust this as best I can to what the ideal outcome would be
    list_lev_score = lev(line, 'Verzeichnis \n')
    
    if yearlymeet_lev_score <= 5:                                                       #I tweak the score until it fits best
        print(f"YOccurence: {line}; Score: {yearlymeet_lev_score}; in Line: {num}")
    elif section_lev_score <= 5:                                                        #I tweak the score until it fits best
        print(f"SOccurence: {line}; Score: {section_lev_score}; in Line: {num}")
    elif list_lev_score <= 5:                                                           #I tweak the score until it fits best
        print(f"LOccurence: {line}; Score: {list_lev_score}; in Line: {num}")
    

YOccurence: Jahresversammlung 
; Score: 0; in Line: 72
SOccurence: 755. Monatssitzung 
; Score: 1; in Line: 349
SOccurence: 756. Monatssitzung 
; Score: 1; in Line: 392
YOccurence: Jahresversammlung 
; Score: 0; in Line: 500
SOccurence: XXX. Monatssitzung 
; Score: 3; in Line: 3167
SOccurence: 757. Monatssitzung 
; Score: 1; in Line: 3249
SOccurence: 758. Monatssitzung 
; Score: 1; in Line: 3364
YOccurence: Jahresversammlung 
; Score: 0; in Line: 4247
SOccurence: 760. Monatssitzung 
; Score: 1; in Line: 4267
SOccurence: 761. Monatssitzung 
; Score: 2; in Line: 4378
SOccurence: 762. Monatssitzung 
; Score: 2; in Line: 4585
SOccurence: 763. Monatssitzung 
; Score: 2; in Line: 4777
SOccurence: 764. Monatssitzung 
; Score: 2; in Line: 4904
SOccurence: 765. Monatssitzung 
; Score: 2; in Line: 5072
SOccurence: 766. Monatssitzung 
; Score: 2; in Line: 5187
SOccurence: 767. Monatssitzung 
; Score: 2; in Line: 5294
LOccurence: Verzeichnis 
; Score: 0; in Line: 8407


In [94]:
#I create a new list with the lines that mark sections being tagged.
#I have to adjust the changes made above

new_text = []

for num, line in enumerate(text, 0):
    
    yearlymeet_lev_score = lev(line,'Jahresversammlung \n')                            
    section_lev_score = lev(line, '750. Monatssitzung \n')                     #I adjust this as best I can to what the ideal outcome would be
    list_lev_score = lev(line, 'Verzeichnis \n')
    
    if yearlymeet_lev_score <= 5:
        ano_line = f"<section type='yearlymeet'> {line}"                 #I open the section tag in the line I detected
        new_text.append(ano_line)
         
    elif section_lev_score <= 5:
        new_text.append("</section> \n")                                 #I close the section tag with an additional line before the new section starts 
        ano_line = f"<section type='monthlymeet'> {line}"
        new_text.append(ano_line)
        
    elif list_lev_score <= 5:
        new_text.append("</section> \n")
        ano_line = f"<section type='list'> {line}"
        new_text.append(ano_line)
        
    else:
        new_text.append(line)
        
new_text.append("</section> \n")                                         #I close the very last section 
        

In [95]:
new_text[330:360]

['*) Der zum Schluss der Sitzung verlesene Jahresbericht für das \n',
 'Jahr 1911 ist in den S.-B. des Jahres auf S. 115 ff. veröffentlicht. \n',
 '8 \n',
 'schwere verzierte silberne Armringe von schöner Arbeit, ein \n',
 'grosser Ring aus dickem gewundenen Golddraht und 11 grosse \n',
 'silberne Ringe, stabrund mit eingestempelten Verzierungen, \n',
 'facettiert oder gewunden, wie sie gewöhnlich als Halsringe ange\xad\n',
 'sprochen werden, aber im vorliegenden Falle eher als Kopf\xad\n',
 'schmuck zu deuten sind. \n',
 'Ein derartiger Fund ist bisher in unseren Provinzen nicht \n',
 'gemacht worden. Nur eine Armbrustfibel von derselben Aus\xad\n',
 'stattung hat sich bei Riga (Plawnekaln) gefunden, doch ähnliche \n',
 'Bronzefibeln bieten Ostpreussen und Finnland. Es handelt sich \n',
 'bei den neben dem Stein gefundenen Sachen nicht um eine Toten\xad\n',
 'beigabe, auch nicht um einen vergrabnen Schatz, da die Gegen\xad\n',
 'stände gleich gemustert sind, meist ganz neu, ohne Spure

In [96]:
#I write the new annotated list into a .txt file 

out_filename = 'Output/GEG/1912-1922_GEG_Ano.txt'                            #Look out for the right year!!

filepath = Path(out_filename)                             
filepath.parent.mkdir(parents=True, exist_ok=True)                       #creates the folders if they don't exist yet

with open(out_filename, 'x') as fp:
    fp.write("".join(str(item) for item in new_text))

### Isolating and annotating membership list 

### Notes:
- In file 1876_GEG_Members_Ano.txt there is no second level homeland/abroad distinction for OM only an in_hometown/outside_hometown distinction. 
- In file 1877_GEG_Members_Ano.txt there is no second level homeland/abroad distinction for OM only an outside_hometown marker. 
- NB: In file 1878_GEG_Members_Ano.txt there are a few persons that were not detected correctly and now share a line with the person before.
- In file 1878_GEG_Members_Ano.txt I manually adapt the wrongly formated persons in line 36 (person 4), line 103 (person 71), line 143 (person 109), and in line 203 (person 26).
- In file 1878_GEG_Members_Ano.txt I manually remove not detected page numbers from line 9, line 66, and line 192. 
- In file 1880_GEG_Members_Ano.txt I remove a wrongly set line break in line 233.
- In file 1881_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 136 (person 101), line 173 (person 138), line 200 (person 22), line 236 (person 58).
- In file 1882_GEG_Members_Ano.txt I manually remove not detected page numbers from line 95, line 119, and line 139.
- In file 1882_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 153 (person 117).
- In file 1883_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 13 (person 9), line 15 (person 11), line 27 (person 22), line 263 (person 69).
- In file 1884_GEG_Members_Ano.txt I manually remove not detected page numbers from line 59, and line 205.
- In file 1884_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 91 (person 58).
- In file 1885_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 9 (person 5), line 12 (person 8), line 129 (person 93-116), line 209 (person 24).
- In file 1886_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 40.
- In file 1887_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 41 (person 35),
- In file 1887_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 58.
- In file 1887_GEG_Members_Ano.txt I manually remove not detected page numbers from line 202.
- In file 1888_GEG_Ano.txt I manually correct the HM homeland label, which was falsely printed as "Im Auslande" in line 8052.
- In file 1888_GEG_Ano.txt I manually correct the CM homeland label, because it would only have been detected with a fuzz.ratio that also detects the abroad tag as a homeland tag in line 8337. 
- In file 1888_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 60.
- In file 1888_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 88 (person 42-71).
- In file 1888_GEG_Members_Ano.txt I manually remove not detected page numbers from line 252, and line 277.
- In file 1889_GEG_Ano.txt I manually add I line break before "Ehrenmitglieder" in line 5210. 
- In file 1889_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 64.
- In file 1889_GEG_Members_Ano.txt I manually remove not detected page numbers from line 206.
- In file 1889_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 254 (person 3-27).
- NB: In file 1890_GEG_Ano.txt the sequence of 1st level membership status information is switched being HM, CM, OM not HM, OM, CM. The code needs to be slightly adjusted.
- In file 1890_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 146.
- In file 1890_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 198 (person 73-96), in line 286 (person 161-162), and in line 325 (person 202).
- NB: In file 1891_GEG_Ano.txt the sequence of 1st level membership status information is switched being HM, CM, OM not HM, OM, CM.
- In file 1891_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 82 (person 43-44), in line 86 (person 45-61), and in line 284 (person 163).
- In file 1891_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 126.
- NB: In file 1892_GEG_Ano.txt the sequence of 1st level membership status information is switched being HM, CM, OM not HM, OM, CM.
- In file 1892_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 61 (person 22-43), in line 85 (person 43-45), in line 134 (person 16-43), in line 164 (person 46-76), and in line 311 (person 191).
- In file 1892_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 134.
- In file 1892_GEG_Members_Ano.txt I manually remove not detected page numbers from line 230.
- NB: In file 1893_GEG_Ano.txt the sequence of 1st level membership status information is switched being HM, CM, OM not HM, OM, CM.
- In file 1893_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 10 (person 6), and in line 167 (person 50-83).
- In file 1893_GEG_Members_Ano.txt I manually remove not detected page numbers from line 36, line 88.
- In file 1893_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 137.
- NB: In file 1894_GEG_Ano.txt the sequence of 1st level membership status information is switched being HM, CM, OM not HM, OM, CM.
- In file 1894_GEG_Ano.txt I manually correct the CM homeland label, because it would only have been detected with a fuzz.ratio that also detects the abroad tag as a homeland tag in line 6121.
- In file 1894_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 67 (person 27), in line 86 (person 44-61), in line 157 (person 40-99), in line 241 (person 122-141), in line 264 (person 145-146),in line 275 (person 156) in line 280 (person 161), and in line 282 (person 163).
- In file 1894_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 129.
- NB: In file 1895_GEG_Ano.txt the sequence of 1st level membership status information is switched being HM, CM, OM not HM, OM, CM.
- In file 1895_GEG_Ano.txt I manually correct the HM abroad tag, because it was too faulty to be detected in line 4359.
- In file 1895_GEG_Members_Ano.txt I manually adapt the wrongly formated person in line 9 (person 5), in line 50 (person 10), and in line 53 (person 13). 
- In file 1895_GEG_Members_Ano.txt I manually remove the note regarding the paid membership fees in line 128.


In [263]:
#I read the file into a list
filename = 'Output/GEG/1895_GEG_Ano.txt'
with open(filename, 'r') as fp:
    text = fp.readlines()
    

In [264]:
#I look for the section markers and write them in lists

sections_s = []
sections_e = []

for num, line in enumerate(text, 0):
    if line.startswith("<section type='list'>"):                           #Only section beginnings with marker "list".
        sections_s.append(num)
        
for num, line in enumerate(text, 1):        
    if line == "</section> \n":                                            #All section endings are the same.
        sections_e.append(num)
        
print(sections_s)
print(sections_e)


[4311, 4762, 5022, 5336]
[957, 1265, 1618, 1938, 2168, 2474, 3093, 3242, 4311, 4762, 5022, 5336, 5473]


In [265]:
#I check which of the list sections is the membership list and create a list only with this section
members_text = text[sections_s[0]:sections_e[-4]]
members_text

["<section type='list'> Verzeichnis \n",
 'der Mitglieder \n',
 'am^chlusse desIahres^8y5. \n',
 'Ehrenmitglieder. \n',
 'Im Jnlande. \n',
 '1) Mag. Nik. Anderson, Docent in Kasan \n',
 '(1892). \n',
 '2) I. R. Aspelin,Staats-Archäolog in Hel-singfors (1887) \n',
 '3) Dr. August Bielen st ein, Pastor in Doblen \n',
 '(1887) \n',
 '4) Landrath Arved v. B r a s ch -Ropkoi (1887j \n',
 'oj Baron Hermann v. Bruiningk, Rit-\n',
 'tersch.-Secretär in Riga (1887) \n',
 '6) Woldemar Graf von dem Broel-Pla-\n',
 'ter auf Dombrowitza in Wolhynien (1876) \n',
 '7) Theodor Baron Bühler, Geh.-Rath, Di-\n',
 'rector des Haupt-Staatsarchivs des Mini-\n',
 'steriums des Auswärtigen in Moskau (1875) \n',
 '8) A. F. Bytschkow, Geh.-Rath, Vice-Prä-stdent der Kais. Archäolog. Gesellschaft in \n',
 'Petersburg (1887) \n',
 '9) Dr. Jak. Hurt, Pastor in Petersburg (1887) \n',
 '10) Michael Kapustin, Geheimrath, Curator \n',
 'des Petersburger Lehrbezirks (1883) \n',
 '11) E. Kunik, Akademiker in Petersburg (

In [266]:
#To create and better visualize the annotation structure I first created a template manually. 

#First round annotating the membership list
#I again use XML inspired tags to mark the structuring elements of the list.
#This is the first level of information including the status of their membership: honorable, ordinary or corresponding.
#Depending on the quality of the OCR these reference words need to be adapted.

members_ano_1 = []
for line in members_text: 
    hon_mem = fuzz.ratio("Ehrenmitglieder", line)                                  #Fuzz works with the lev function, but makes it way faster.
    ord_mem = fuzz.ratio("Ordentliche Mitglieder", line)                           #I had difficulties installing fuzzywuzzy in jupyter labs and therefore didn't use it earlier. 
    cor_mem = fuzz.ratio("Correspondierende Mitglieder", line)
    
    if hon_mem >= 90:
        members_ano_1.append("<members>")                                          #I open members section
        members_ano_1.append(f"<honorable_members> {line}")                        #I open honorable members section
        print("HM detected")
    
    elif ord_mem >= 80:
        #members_ano_1.append("</honorable_members>")                               #I close honorable members section
        members_ano_1.append("</corresponding_members>")
        members_ano_1.append(f"<ordinary_members> {line}")                         #I open ordinary members section
        print("OM detected")
        
    elif cor_mem >=90:
        #members_ano_1.append("</ordinary_members>")                               #I close ordinary members section
        members_ano_1.append("</honorable_members>")
        members_ano_1.append(f"<corresponding_members> {line}")                    #I open corresponding members section
        print("CM detected")
        
    elif line == "</section> \n":
        #members_ano_1.append("</corresponding_members>")                           #I close corresponding members section
        members_ano_1.append("</ordinary_members>")
        members_ano_1.append("</members>")                                         #I close members section
        members_ano_1.append(line)
        print("Section tag detected")
    
    else:                                                                          #Don't forget to also append the rest of the original list
        members_ano_1.append(line)
        
members_ano_1

HM detected
CM detected
OM detected
Section tag detected


["<section type='list'> Verzeichnis \n",
 'der Mitglieder \n',
 'am^chlusse desIahres^8y5. \n',
 '<members>',
 '<honorable_members> Ehrenmitglieder. \n',
 'Im Jnlande. \n',
 '1) Mag. Nik. Anderson, Docent in Kasan \n',
 '(1892). \n',
 '2) I. R. Aspelin,Staats-Archäolog in Hel-singfors (1887) \n',
 '3) Dr. August Bielen st ein, Pastor in Doblen \n',
 '(1887) \n',
 '4) Landrath Arved v. B r a s ch -Ropkoi (1887j \n',
 'oj Baron Hermann v. Bruiningk, Rit-\n',
 'tersch.-Secretär in Riga (1887) \n',
 '6) Woldemar Graf von dem Broel-Pla-\n',
 'ter auf Dombrowitza in Wolhynien (1876) \n',
 '7) Theodor Baron Bühler, Geh.-Rath, Di-\n',
 'rector des Haupt-Staatsarchivs des Mini-\n',
 'steriums des Auswärtigen in Moskau (1875) \n',
 '8) A. F. Bytschkow, Geh.-Rath, Vice-Prä-stdent der Kais. Archäolog. Gesellschaft in \n',
 'Petersburg (1887) \n',
 '9) Dr. Jak. Hurt, Pastor in Petersburg (1887) \n',
 '10) Michael Kapustin, Geheimrath, Curator \n',
 'des Petersburger Lehrbezirks (1883) \n',
 '11) E.

In [267]:
#I determine where the first level structural elements start and end, 
#because I want the lower levels to reference the parent in order to avoid a duplicate in a marker opener.

for num, line in enumerate(members_ano_1, 0):
    if line.startswith("<honorable_members>"):                    #The opening marker gives the number for the starting point of the iteration of the list.
        start_hm = num
    if line == "</honorable_members>":                            #The closing marker gives the number for the end point of the iteration of the list.
        end_hm = num +1                                           #I add 1 because I want the closing line itself also included in the iterated section.            
        
for num, line in enumerate(members_ano_1, 0):
    if line.startswith("<ordinary_members>"):
        start_om = num
    if line == "</ordinary_members>":
        end_om = num +1

for num, line in enumerate(members_ano_1, 0):
    if line.startswith("<corresponding_members>"):
        start_cm = num
    if line == "</corresponding_members>":
        end_cm = num +1        

#members_ano_1[start_om:end_om]

In [268]:
#Second round annotating the membership list
#This is the second level of information including the location of the member as inside or outside of the Russian Empire.
#Since this distinction is made for each of the first level structural elements I want to uniquely describe the opening markers.

members_ano_2 = []

for num, line in enumerate(members_ano_1, 0):
    if num in range(start_hm, end_hm):                                   #I only go through the range of the honorable members section.

        home_mem = fuzz.ratio("Im Inlande. ", line)                 
        abroad_mem = fuzz.ratio("Im Auslande. ", line)

        if home_mem >= 80:
            members_ano_2.append(f"<homeland parent='HM'> {line}")       #I open the homeland section for honorable members
            print("HM homeland detected")

        elif abroad_mem >= 70:
            members_ano_2.append("</homeland>")                          #I close the homeland section 
            members_ano_2.append(f"<abroad parent='HM'> {line}")         #I open the abroad section for honorable members
            print("HM abroad detected")

        elif line == "</honorable_members>":
            members_ano_2.append("</abroad>")                            #I close the abroad section 
            members_ano_2.append(line)
            print("HM tag detected")
            
        else:
            members_ano_2.append(line)                                   #Don't forget to append the lines in between. 
    
    elif num in range(start_om, end_om):                                 #I only go through the range of the ordinary members section.

        #home_mem = fuzz.ratio("Im Inlande. ", line)
        #abroad_mem = fuzz.ratio("Im Auslande. ", line)
        in_mem = fuzz.ratio("I. In Dorpat. ", line)
        out_mem = fuzz.ratio("II. Außerhalb Dorpat. ", line)

        if in_mem >= 70:
            members_ano_2.append(f"<in_hometown> {line}")                 #I open the in hometown section
            print("OM In detected")

        elif out_mem >= 80:
            members_ano_2.append("</in_hometown>")                        #I close the in hometown section 
            members_ano_2.append(f"<outside_hometown> {line}")
            print("OM Out detected")

        #if home_mem >= 80:
            #members_ano_2.append(f"<homeland parent='OM'> {line}")       #I open the homeland section for the ordinary members

        #elif abroad_mem >= 80:
           # members_ano_2.append("</homeland>")                          #I close the homeland section 
            #members_ano_2.append(f"<abroad parent='OM'> {line}")         #I open the abroad section for the ordinary members

        elif line == "</ordinary_members>":     
            #members_ano_2.append("</abroad>")                             #I close the abroad section
            members_ano_2.append("</outside_hometown>")                   #I close the outside hometown section
            members_ano_2.append(line)
            print("OM tag detected")
            
        else:
            members_ano_2.append(line)
    
    elif num in range(start_cm, end_cm):                                 #I only go through the range of the corresponding members section.

        home_mem = fuzz.ratio("Im Inlande. ", line)
        abroad_mem = fuzz.ratio("Im Auslande. ", line)

        if home_mem >= 80:
            members_ano_2.append(f"<homeland parent='CM'> {line}")       #I open the homeland section for the corresponding members
            print("CM homeland detected")

        elif abroad_mem >= 80:
            members_ano_2.append("</homeland>")                          #I close the homeland section 
            members_ano_2.append(f"<abroad parent='CM'> {line}")         #I open the abroad section for the corresponding members
            print("CM abroad detected")

        elif line == "</corresponding_members>":
            members_ano_2.append("</abroad>")                            #I close the abroad section 
            members_ano_2.append(line)
            print("CM tag detected")
            
        else:
            members_ano_2.append(line)
    
    
    else:
        members_ano_2.append(line)                                       #Don't forget to also append the lines outside of the sections.

members_ano_2

HM homeland detected
HM abroad detected
HM tag detected
CM homeland detected
CM abroad detected
CM tag detected
OM In detected
OM Out detected
OM tag detected


["<section type='list'> Verzeichnis \n",
 'der Mitglieder \n',
 'am^chlusse desIahres^8y5. \n',
 '<members>',
 '<honorable_members> Ehrenmitglieder. \n',
 "<homeland parent='HM'> Im Jnlande. \n",
 '1) Mag. Nik. Anderson, Docent in Kasan \n',
 '(1892). \n',
 '2) I. R. Aspelin,Staats-Archäolog in Hel-singfors (1887) \n',
 '3) Dr. August Bielen st ein, Pastor in Doblen \n',
 '(1887) \n',
 '4) Landrath Arved v. B r a s ch -Ropkoi (1887j \n',
 'oj Baron Hermann v. Bruiningk, Rit-\n',
 'tersch.-Secretär in Riga (1887) \n',
 '6) Woldemar Graf von dem Broel-Pla-\n',
 'ter auf Dombrowitza in Wolhynien (1876) \n',
 '7) Theodor Baron Bühler, Geh.-Rath, Di-\n',
 'rector des Haupt-Staatsarchivs des Mini-\n',
 'steriums des Auswärtigen in Moskau (1875) \n',
 '8) A. F. Bytschkow, Geh.-Rath, Vice-Prä-stdent der Kais. Archäolog. Gesellschaft in \n',
 'Petersburg (1887) \n',
 '9) Dr. Jak. Hurt, Pastor in Petersburg (1887) \n',
 '10) Michael Kapustin, Geheimrath, Curator \n',
 'des Petersburger Lehrbezir

In [ ]:
#Skip if no 3rd Level of Membership Status

#I get the start and end point of the homeland section of ordinary members

for num, line in enumerate(members_ano_2, 0):
    if line.startswith("<homeland parent='OM'>"):
        start_hl = num
    if line.startswith("<abroad parent='OM'>"):           #Since the closing section is the same for all membership status section I have to use the next best unique identifier
        end_hl = num                                      #Now with out the added 1 so I only get the list until the closing mark 
        
        
#members_ano_2[start_hl:end_hl]

In [ ]:
#Skip if no 3rd Level of Membership Status

#Third round annotating the membership list. 
#Only the homeland section for the ordinary members further destinguishes between people living within Dorpat/Riga or outside of it. 


members_ano_3 = []

for num, line in enumerate(members_ano_2, 0):
    if num in range(start_hl, end_hl):                                    #Only the range of homeland section for ordinary members

        in_mem = fuzz.ratio("I. In Dorpat. ", line)
        out_mem = fuzz.ratio("II. Außerhalb Dorpat. ", line)

        if in_mem >= 90:
            members_ano_3.append(f"<in_hometown> {line}")                 #I open the in hometown section 

        elif out_mem >= 90:
            members_ano_3.append("</in_hometown>")                        #I close the in hometown section 
            members_ano_3.append(f"<outside_hometown> {line}")            #I open the outside hometown section 

        elif line == "</homeland>":
            members_ano_3.append("</outside_hometown>")                   #I close the outside hometown section 
            members_ano_3.append(line)
            
        else:
            members_ano_3.append(line)
    
    else:
        members_ano_3.append(line)
        
members_ano_3

In [269]:
#First step of the cleaning process of the membership list
#Adjust Ano numbers in case third level structure was skipped

members_ano_3 = []

for line in members_ano_2:
    if line.startswith("— "):                      #I remove the page number markers 
        members_ano_2.remove(line)

for line in members_ano_2:
    new_line = re.sub("\n", "", line)             #I remove the line breaks to reconnect information about one person that is split in two lines.
    members_ano_3.append(new_line)

        
#members_ano_3

In [270]:
#Second step of the cleaning process of the membership list
#NB! This is only possible if all members have a clear marker at the beginning --> Some files have OCR errors here. 

members_ano_4 = []

for line in members_ano_3:
    if line.startswith("<"):
        new_line = re.sub(r"<", r"\n<", line)             #I insert a line break before each marker
        members_ano_4.append(new_line)
    else:
        new_line = re.sub(r"^(\S{1,3})\)", r"\n\1)", line)    #I insert a number before each number signifying a person 
        members_ano_4.append(new_line)
        
#members_ano_4

In [271]:
out_filename = 'Output/GEG/1895_GEG_Members_Ano.txt'                #Look out for the right year!!

filepath = Path(out_filename)                             
filepath.parent.mkdir(parents=True, exist_ok=True)                  #creates the folders if they don't exist yet

with open(out_filename, 'x') as fp:
    fp.write("".join(str(item) for item in members_ano_4))          #By joining the list elements without any symbol the 
                                                                    #line breaks will be at the end of each line the next time I read the document in 